In [83]:
# from __future__ import print_fuction
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [84]:
batchsize = 64
imagesize = 64

In [85]:
transform= transforms.Compose([transforms.Scale(imagesize), transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),])
dataset = dset.CIFAR10(root='./data', download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchsize, shuffle=True, num_workers=2)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


Files already downloaded and verified


In [86]:
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Conv') != -1:
    m.weight.data.normal_(0.0,0.02)
  elif classname.find('BatchNorm') != -1:
    m.weight.data.normal_(1.0,0.02)
    m.bias.data.fill_(0)

In [ ]:
#defining Generator
class G(nn.Module):
  def __init__(self):
    super(G,self).__init__()
    self.main = nn.Sequential(
        nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
        nn.BatchNorm2d(512),
        nn.ReLU(True),
        nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),
        nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
        nn.Tanh()
    )
  def forward(self, input):
    output = self.main(input)
    return output
  
#creating the generator
netG = G()
netG.apply(weights_init)

In [75]:
#defining Discriminator
class D(nn.Module):
  def __init__(self):
   super(D, self).__init__()
   self.main = nn.Sequential(
       nn.Conv2d(3, 64, 4, 2, 1, bias=False),
       nn.LeakyReLU(0.2, inplace=True),
       nn.Conv2d(64, 128, 4, 2, 1, bias=False),
       nn.BatchNorm2d(128),
       nn.LeakyReLU(0.2, inplace=True),
       nn.Conv2d(128, 256, 4, 2, 1, bias=False),
       nn.BatchNorm2d(256),
       nn.LeakyReLU(0.2, inplace=True),
       nn.Conv2d(256, 512, 4, 2, 1, bias=False),
       nn.BatchNorm2d(512),
       nn.LeakyReLU(0.2, inplace=True),
       nn.Conv2d(512, 1, 4, 1, 0, bias=False),
       nn.Sigmoid()
   )
  def forward(self, input):
    output = self.main(input)
    return output.view(-1)#step6

In [ ]:
#creating the discriminator
netD = D()
netD.apply(weights_init)



#training the DCGANs
criterian = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

for epoch in range(25):
  for i, data in enumerate(dataloader, 0):
    #updating weights of discrimi.
    netD.zero_grad()
    #training the discri. on real images
    real,_ = data
    input = Variable(real)
    target = Variable(torch.ones(input.size()[0]))
    output = netD(input)
    errD_real = criterian(output, target)
    
    #training the discri. on fake images
    noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
    fake = netG(noise)
    target = Variable(torch.zeros(input.size()[0]))
    output = netD(fake.detach())
    errD_fake = criterian(output, target)
    
    #backpropagating error
    errD = errD_real + errD_fake
    errD.backward()
    optimizerD.step()
    
    #updating the weights for generator
    netG.zero_grad()
    target = Variable(torch.ones(input.size()[0]))
    output = netD(fake)
    errG = criterian(output, target)
    errG.backward()
    optimizerG.step()

    print('[%d%d][%d%d] LossD: %.4f LossG: %.4f' % (epoch, 25, i, len(dataloader), errD, errG))
    if i%100==0:
      vutils.save_image(real, '%s/real_samples.png' % "/content/drive/MyDrive/Udemy learning path/A-Z AI- openCV, SSD, GANs/mod3- image creation with GANs/mere_results", normalize=True)
      fake = netG(noise)
      vutils.save_image(fake.data, '%s/fake_samples_epochs_%03d.png' % ("/content/drive/MyDrive/Udemy learning path/A-Z AI- openCV, SSD, GANs/mod3- image creation with GANs/mere_results", epoch), normalize=True)